![](https://media.istockphoto.com/id/1362135301/vector/timetable-board.jpg?s=612x612&w=0&k=20&c=V4YZ1g1srAYCjSdwcKgCQ6kSDmY_fqMLJBPUNbJXXX0=)

[Link](https://github.com/the-faisalahmed/Optimization/blob/main/ISE-3700%20Airport%20Service%20Management.doc) to doc file breaking down the objectives of this project.

In [1]:
#!pip install ortools
from ortools.sat.python import cp_model

In [2]:
import matplotlib.pyplot as plt
import itertools
import math
import numpy as np
from datetime import datetime, time, timedelta

In [3]:
Flights = {
"AAL2561":  {"Target":"18:39:30", "Dept":"WILEY"},
"DAL4703":	{"Target":"18:38:15", "Dept":"WILEY"},
"DAL2684":	{"Target":"18:37:00",	"Dept":"WILEY"},
"AAL3441":	{"Target":"18:36:00",	"Dept":"CHASZ"},
"AAL0639":	{"Target":"18:35:00",	"Dept":"WILEY"},
"EGF1218":	{"Target":"18:33:00",	"Dept":"NELYN"},
"DAL2777":	{"Target":"18:32:00",	"Dept":"WILEY"},
"AAL9355":	{"Target":"18:30:00",	"Dept":"SMITH"},
"AAL7665":	{"Target":"18:29:00",	"Dept":"WILEY"},
"EGF9310":	{"Target":"18:27:00",	"Dept":"CHASZ"},
"AAL7665":	{"Target":"18:26:00",	"Dept":"WILEY"},
"DAL9416":	{"Target":"18:24:00",	"Dept":"PHILS"},
"AAL8912":	{"Target":"18:23:00",	"Dept":"WILEY"},
"DAL5519":	{"Target":"18:21:00",	"Dept":"NELYN"},
"EGF0862":	{"Target":"18:20:00",	"Dept":"WILEY"},
"SKW3013":	{"Target":"18:18:00",	"Dept":"NIKKY"},
"EGF1050":	{"Target":"18:17:00",	"Dept":"WILEY"},
"AAL1180":	{"Target":"18:15:00",	"Dept":"NELYN"},
"KAL0043":	{"Target":"18:14:00",	"Dept":"WILEY"},
"AAL7799":	{"Target":"18:12:00",	"Dept":"WORTH"},
"DAL5919":	{"Target":"18:11:00",	"Dept":"WILEY"},
"AAL3074":	{"Target":"18:10:00",	"Dept":"WORTH"},
"EGF0512":	{"Target":"18:09:00",	"Dept":"CHASZ"},
"AAL3074":	{"Target":"18:08:00",	"Dept":"SMITH"},
"AAL8365":	{"Target":"18:07:00",	"Dept":"WILEY"},
"DAL2964":	{"Target":"18:05:00",	"Dept":"WHALT"},
"AAL2435":	{"Target":"18:04:00",	"Dept":"WILEY"},
"AAL3546":	{"Target":"18:02:00",	"Dept":"WORTH"},
"DAL5917":	{"Target":"18:01:00",	"Dept":"WILEY"},
"DAL2306":	{"Target":"18:00:00",	"Dept":"PHILS"},
}

Dept_Fix = {
'NOBLR': {'Direction': 'N', 'Neighbor': ['NOBLR', 'NELYN'], 'MIT': 3},
'NELYN': {'Direction': 'N', 'Neighbor': ['NELYN', 'NOBLR', 'NIKKY'],'MIT': 3},
'NIKKY': {'Direction': 'N','Neighbor': ['NIKKY', 'NELYN', 'KNAVE'],'MIT': 3},
'KNAVE': {'Direction': 'N','Neighbor': ['KNAVE', 'NIKKY', 'NEXAR'],'MIT': 3},
'NEXAR': {'Direction': 'N', 'Neighbor': ['NEXAR', 'KNAVE'], 'MIT': 3},
'PHILS': {'Direction': 'S', 'Neighbor': ['PHILS', 'SMITH'], 'MIT': 3},
'SMITH': {'Direction': 'S','Neighbor': ['SMITH', 'PHILS', 'CHASZ'],'MIT': 3},
'CHASZ': {'Direction': 'S','Neighbor': ['CHASZ', 'SMITH', 'ALISA'],'MIT': 3},
'ALISA': {'Direction': 'S','Neighbor': ['ALISA', 'CHASZ', 'EVANS'],'MIT': 3},
'EVANS': {'Direction': 'S', 'Neighbor': ['EVANS', 'ALISA'], 'MIT': 3},
'BEATY': {'Direction': 'E', 'Neighbor': ['BEATY', 'EARTZ'], 'MIT': 3},
'EARTZ': {'Direction': 'E','Neighbor': ['EARTZ', 'BEATY', 'BEAMR'],'MIT': 3},
'BEAMR': {'Direction': 'E','Neighbor': ['BEAMR', 'EARTZ', 'TREAT'],'MIT': 3},
'TREAT': {'Direction': 'E','Neighbor': ['TREAT', 'BEAMR', 'CREAK'],'MIT': 3},
'CREAK': {'Direction': 'E', 'Neighbor': ['CREAK', 'TREAT'], 'MIT': 3},
'WILEY': {'Direction': 'W', 'Neighbor': ['WILEY', 'WICKR'], 'MIT': 3},
'WICKR': {'Direction': 'W','Neighbor': ['WICKR', 'WILEY', 'WORTH'],'MIT': 12},
'WORTH': {'Direction': 'W','Neighbor': ['WORTH', 'WICKR', 'WHALT'],'MIT': 3},
'WHALT': {'Direction': 'W','Neighbor': ['WHALT', 'WORTH', 'WYMON'],'MIT': 3},
'WYMON': {'Direction': 'W', 'Neighbor': ['WYMON', 'WHALT'], 'MIT': 3}
}

def timediff(x):
    a = datetime.strptime(x, '%H:%M:%S')
    b = datetime.strptime('18:00:00', '%H:%M:%S')
    return ((a - b).total_seconds()/60)

for i in Flights.keys():
   Flights[i]['Target'] = timediff(Flights[i]['Target'])

Flights

{'AAL2561': {'Target': 39.5, 'Dept': 'WILEY'},
 'DAL4703': {'Target': 38.25, 'Dept': 'WILEY'},
 'DAL2684': {'Target': 37.0, 'Dept': 'WILEY'},
 'AAL3441': {'Target': 36.0, 'Dept': 'CHASZ'},
 'AAL0639': {'Target': 35.0, 'Dept': 'WILEY'},
 'EGF1218': {'Target': 33.0, 'Dept': 'NELYN'},
 'DAL2777': {'Target': 32.0, 'Dept': 'WILEY'},
 'AAL9355': {'Target': 30.0, 'Dept': 'SMITH'},
 'AAL7665': {'Target': 26.0, 'Dept': 'WILEY'},
 'EGF9310': {'Target': 27.0, 'Dept': 'CHASZ'},
 'DAL9416': {'Target': 24.0, 'Dept': 'PHILS'},
 'AAL8912': {'Target': 23.0, 'Dept': 'WILEY'},
 'DAL5519': {'Target': 21.0, 'Dept': 'NELYN'},
 'EGF0862': {'Target': 20.0, 'Dept': 'WILEY'},
 'SKW3013': {'Target': 18.0, 'Dept': 'NIKKY'},
 'EGF1050': {'Target': 17.0, 'Dept': 'WILEY'},
 'AAL1180': {'Target': 15.0, 'Dept': 'NELYN'},
 'KAL0043': {'Target': 14.0, 'Dept': 'WILEY'},
 'AAL7799': {'Target': 12.0, 'Dept': 'WORTH'},
 'DAL5919': {'Target': 11.0, 'Dept': 'WILEY'},
 'AAL3074': {'Target': 8.0, 'Dept': 'SMITH'},
 'EGF0512': {

In [4]:
# Creates the model and set solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

total_minutes = 60

# Flight i uses dept fix j at time k minutes past 1800
x = {(i,j):model.NewBoolVar(f"x_{i}_{j}") \
     for i in Flights.keys() for j in Dept_Fix.keys()}
     
# Number of minutes past 18:00 that flight i will depart
y = {(i):model.NewIntVar(0,100*total_minutes,f"y_{i}") \
     for i in Flights.keys()}
y_abs = {(i):model.NewIntVar(0,100*total_minutes,f"y_abs_{i}") \
     for i in Flights.keys()}
z_abs = {(i):model.NewIntVar(0,100*total_minutes,f"z_abs_{i}") \
     for i in range(math.comb(len(Flights.keys()),2))}
d_abs = {(i):model.NewIntVar(0,100*total_minutes,f"d_abs_{i}") \
     for i in range(len([((i,j),(a,b)) \
     for (i,j),(a,b) in list(itertools.combinations(x.keys(),2)) if i!=a and j==b]))}

# Each flight must leave outside of one fix

for i in Flights.keys():
     model.AddExactlyOne([x[i,j] for j in Dept_Fix.keys() \
                    if j in Dept_Fix[Flights[i]['Dept']]['Neighbor']])

# No flights leave at the same time

model.AddAllDifferent([y[i] for i in Flights.keys()])

# Can't change target off time by more than +- 5 minutes

for (i,j),v in x.items():
     model.AddAbsEquality(y_abs[i],y[i] - int(100*Flights[i]['Target']))
     model.Add(y_abs[i]<=100*5).OnlyEnforceIf(v)
     # WICKR closed from 1800 to 1835    
     if j == 'WICKR':
          model.Add(y[i] >= 100*35).OnlyEnforceIf(v)

# No two flights within 1 minute of one another

count = 0
for i,j in list(itertools.combinations(Flights.keys(),2)):
     model.AddAbsEquality(z_abs[count], y[i] - y[j])
     model.Add(z_abs[count] >= 100)
     count+=1

# 75 second delay if 2 flights using the same fixes and no MIT restrictions for either
# 180 seconds for flights departing via WILEY from 1800-1835

count = 0
complete_list = list(itertools.combinations(x.keys(),2))

for (i,j),(a,b) in [((i,j),(a,b)) for (i,j),(a,b) in complete_list if i!=a and j==b]:
     model.AddAbsEquality(d_abs[count], y[i] - y[a])
     # if j == 'WILEY':
     #      model.Add(d_abs[count] >= int(100*(180/60)))
     #      # pass
     # else:
     model.Add(d_abs[count] >= int(100*(75/60))).OnlyEnforceIf([x[i,j],x[a,b]])
     count+=1

# Can't reroute within 10 min of target off time

for i in Flights.keys():
     for j in Dept_Fix.keys():
          if j == Flights[i]['Dept'] and int(Flights[i]['Target']) <= 10:
               model.Add(x[i,j] == 1)

non_reroute = [i for i in Flights.keys() if Flights[i]['Target'] <= 10]

for i in range(len(non_reroute)):
     if non_reroute[i] != non_reroute[-1]:
          model.Add(y[non_reroute[i]] >= y[non_reroute[i+1]])

# Minimize throughput
model.Minimize(sum(y[i] for i in y.keys()))
status = solver.Solve(model)
print(f"Status = {solver.StatusName(status)}")

Status = OPTIMAL


In [6]:
sol = sorted([[i, Flights[i]['Target'], Flights[i]['Dept'], '------>',i,solver.value(y[i])/100, j ] \
               for i,j in x.keys() if solver.value(x[i,j]) == 1], key=lambda x:x[5],reverse=True)
sol

[['AAL2561', 39.5, 'WILEY', '------>', 'AAL2561', 34.5, 'WILEY'],
 ['DAL4703', 38.25, 'WILEY', '------>', 'DAL4703', 33.25, 'WILEY'],
 ['DAL2684', 37.0, 'WILEY', '------>', 'DAL2684', 32.0, 'WILEY'],
 ['AAL3441', 36.0, 'CHASZ', '------>', 'AAL3441', 31.0, 'ALISA'],
 ['AAL0639', 35.0, 'WILEY', '------>', 'AAL0639', 30.0, 'WILEY'],
 ['EGF1218', 33.0, 'NELYN', '------>', 'EGF1218', 28.0, 'NOBLR'],
 ['DAL2777', 32.0, 'WILEY', '------>', 'DAL2777', 27.0, 'WILEY'],
 ['AAL9355', 30.0, 'SMITH', '------>', 'AAL9355', 25.0, 'PHILS'],
 ['EGF9310', 27.0, 'CHASZ', '------>', 'EGF9310', 22.0, 'ALISA'],
 ['AAL7665', 26.0, 'WILEY', '------>', 'AAL7665', 21.0, 'WILEY'],
 ['DAL9416', 24.0, 'PHILS', '------>', 'DAL9416', 19.0, 'PHILS'],
 ['AAL8912', 23.0, 'WILEY', '------>', 'AAL8912', 18.0, 'WILEY'],
 ['DAL5519', 21.0, 'NELYN', '------>', 'DAL5519', 16.0, 'NOBLR'],
 ['EGF0862', 20.0, 'WILEY', '------>', 'EGF0862', 15.0, 'WILEY'],
 ['SKW3013', 18.0, 'NIKKY', '------>', 'SKW3013', 13.0, 'KNAVE'],
 ['EGF10

In [7]:
sol = [f'Flight {i} leaves {j} at {solver.value(y[i])/100} minutes past 1800. Originally leaving {Flights[i]["Dept"]} {Flights[i]["Target"]} minutes past 1800.' for i,j in x.keys() if solver.value(x[i,j]) == 1]
sol

['Flight AAL2561 leaves WILEY at 34.5 minutes past 1800. Originally leaving WILEY 39.5 minutes past 1800.',
 'Flight DAL4703 leaves WILEY at 33.25 minutes past 1800. Originally leaving WILEY 38.25 minutes past 1800.',
 'Flight DAL2684 leaves WILEY at 32.0 minutes past 1800. Originally leaving WILEY 37.0 minutes past 1800.',
 'Flight AAL3441 leaves ALISA at 31.0 minutes past 1800. Originally leaving CHASZ 36.0 minutes past 1800.',
 'Flight AAL0639 leaves WILEY at 30.0 minutes past 1800. Originally leaving WILEY 35.0 minutes past 1800.',
 'Flight EGF1218 leaves NOBLR at 28.0 minutes past 1800. Originally leaving NELYN 33.0 minutes past 1800.',
 'Flight DAL2777 leaves WILEY at 27.0 minutes past 1800. Originally leaving WILEY 32.0 minutes past 1800.',
 'Flight AAL9355 leaves PHILS at 25.0 minutes past 1800. Originally leaving SMITH 30.0 minutes past 1800.',
 'Flight AAL7665 leaves WILEY at 21.0 minutes past 1800. Originally leaving WILEY 26.0 minutes past 1800.',
 'Flight EGF9310 leaves AL